In [4]:
# 1. Imports
import torch
import torchvision.transforms as transforms
from PIL import Image
import io

# 2. Function: JPEG Compression Defence
def jpeg_defence(image_tensor, quality=75):
    to_pil = transforms.ToPILImage()
    to_tensor = transforms.ToTensor()

    image = to_pil(image_tensor.squeeze().cpu())
    buffer = io.BytesIO()
    image.save(buffer, format='JPEG', quality=quality)
    buffer.seek(0)
    compressed_image = Image.open(buffer)
    return to_tensor(compressed_image).unsqueeze(0).to(device)

# 3. Apply JPEG Defence to Perturbed Image
jpeg_image = jpeg_defence(perturbed_image)

# 4. Evaluate performance
model.eval()
with torch.no_grad():
    output_jpeg = model(jpeg_image)
    pred_jpeg = output_jpeg.argmax(dim=1)

print(f"JPEG Defence prediction: {lfw_dataset.classes[pred_jpeg.item()]}")


NameError: name 'perturbed_image' is not defined